# Determining a Pareto Efficient Set

Resources:

https://stackoverflow.com/questions/32791911/fast-calculation-of-pareto-front-in-python

http://code.activestate.com/recipes/578287-multidimensional-pareto-front/

https://oapackage.readthedocs.io/en/latest/examples/example_pareto.html

## Import

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import timeit

## Functions

Note: The pareto sets for these functions are searching for the higest, not lowest, objective function values. Signs need to be flipped if searching for the lowest values.

### Pareto Set Comparison

In [ ]:
def compare_pareto_sets(set1,set2):
    '''
    Function which compares two pareto sets *Only works if the pareto sets have the same order
    
    Inputs:
    set1, set2 -- pareto set
    
    Output:
    similarity -- if True, pareto sets are the same
    '''
    compare = set1 == set2
#     print(compare)
    similar = compare.all()
    return similar

### Dominates

In [ ]:
def dominates(row, candidateRow):
    '''
    Function to determine if a set of costs (objective function values) "dominates" or is better than another row of costs.
    
    Inputs:
    row -- set of multiple costs to compare against
    candidate row -- set of multiple costs that is compared to row to see if it is better
    
    Output:
    True/False
        -if True: each cost value of the row set is greater than or equal to the candidate row set; the row set dominates
        -if False: at least one of the cost values of the row set is smaller than the candidate row set; the candidate row set dominates
    '''
    return sum([row[x] >= candidateRow[x] for x in range(len(row))]) == len(row) 



#### Test: dominates

Change the row and candidateRow sets to test. 


In [ ]:
row = [0,0,0]
candidateRow = [0,0,0]
dominates(row,candidateRow)

### Simple Cull

Finds the pareto set

*add a line which automatically puts the sets of costs in which one of the costs is the best out of all of that type of cost in the pareto set?
--I don't think this is necessary

*just because a row may be worse than a candidate row does not mean it is outside the pareto set?

*THis is when looking for the highest cost value in a set

In [ ]:
def simple_cull(inputPoints, dominates):
    '''
    Function which takes evaluates lists of multiple costs against eachother to determine a pareto optimal set
    
    Inputs:
    inputPoints --  list of lists of cost values
    dominates -- the dominates function
    
    Output:
    pareto points -- list of lists that are in the pareto  (dominant) set
    dominated points -- list of lists that are in the dominated set (not pareto optimal)
    
    '''
    
    #Make an empty set in which to put the pareto optimal cost sets
    paretoPoints = set()
    #Initialize candidate row
    candidateRowNr = 0
    #Make an empty set in which to put the dominated cost sets
    dominatedPoints = set()
    
    while True:
        #Grab candidate row from the input list of cost sets and remove it from the input list for testing
        candidateRow = inputPoints[candidateRowNr]
        inputPoints.remove(candidateRow)
        #Initialize row number which will be compared with candidate row
        rowNr = 0
        
        #Flag indicating the candidate set has not been dominated 
        nonDominated = True
        
        #while loop: ends when there are no more input points (all the input points have been placed in either the pareto or dominated sets) and the row the entire input sets has not been enumerated through
        while len(inputPoints) != 0 and rowNr < len(inputPoints):
            #make comparison row the row number the loop is on
            row = inputPoints[rowNr]
            
            #apply dominates function
            if dominates(candidateRow, row): #If this is true: candidate row costs are all better than the row compared against
                # If it is worse on all features remove the row from the input array and place in dominated set
                # this indicates the row set is dominated
                inputPoints.remove(row)
                dominatedPoints.add(tuple(row))
            
            elif dominates(row, candidateRow): #if this is true: the row costs are all better than the candidate row costs compared against
                #this indicates the candidate set is dominated, end this loop and pick a new candidate row
                nonDominated = False
                dominatedPoints.add(tuple(candidateRow))
                rowNr += 1
            
            else: #if both of the above are false, neither the candidate row nor the comparison are dominated, so leave the row in the input points and compare the candidate row to the next row
                rowNr += 1

        if nonDominated:
            # add the non-dominated point to the Pareto frontier
            paretoPoints.add(tuple(candidateRow))
        
        # if there are no more input rows to compare
        if len(inputPoints) == 0:
            break
    
    return paretoPoints, dominatedPoints

#### Test: simple cull

Give a set of input points, can change to test. 

In [ ]:
inputPoints = [[1,0,0],[0,1,0],[0,0,1],[1,0,1]]
paretoPoints, dominatedPoints = simple_cull(inputPoints, dominates)

In [ ]:
paretoPoints

In [ ]:
dominatedPoints

Test on 2d data

In [ ]:
#X cost sets with two types of cost/set
data=np.random.rand(2,10)
plt.plot(data[0,:],data[1,:],'.b',label='Original Data')
plt.xlabel('Objective 1')
plt.ylabel('Object 2')
plt.legend()

In [ ]:
data = data.T
data = data.tolist()

In [ ]:
paretoPoints, dominatedPoints = simple_cull(data, dominates)

In [ ]:
paretoPoints

In [ ]:
dominatedPoints

In [ ]:
paretoPoints = np.array(list(paretoPoints))
dominatedPoints = np.array(list(dominatedPoints))

In [ ]:
plt.plot(paretoPoints[:,0],paretoPoints[:,1],'.r',label='Pareto Points')
plt.plot(dominatedPoints[:,0],dominatedPoints[:,1],'.b',label='Dominated Points')
plt.xlabel('Objective 1')
plt.ylabel('Object 2')
plt.legend()

Observations:

1. For two costs, runs quickly for 1000 points

Test on higher dimensional data

In [ ]:
data=np.random.rand(4,10000)
data = data.T
data = data.tolist()

In [ ]:
paretoPoints, dominatedPoints = simple_cull(data, dominates)

Observations:
    
    1. For 4 costs, 10000 points, simple cull still runs quickly (~<1s)
    2. Takes a lot more time with 100K points (about a minute?)
    3. Need to test out efficiency opportunities

### Is_pareto_efficient_dumb
*This is when looking for the highest cost value in a set

In [ ]:
def is_pareto_efficient_dumb(costs):
    '''
    Function to find the pareto-efficient points, fast for many costs, but slow for many datapoints
    
    Inputs:
    costs -- An (n_points, n_costs) array
    
    Output:
    is_efficient -- A (n_points, ) boolean array, indicating whether each point is Pareto efficient
    '''
    #Make an output array
    is_efficient = np.ones(costs.shape[0], dtype = bool)
    
    for i, c in enumerate(costs):
        #for position i in the is_efficient output set
        #costs[:i] -- all of the rows of costs leading up to row i
        #costs[:i]>c -- check whether for all of the rows leading up to row i, a cost in a given row is bigger than a cost c in the row i
        #np.any(costs[:i]>c,axis=1) -- evaluates this for any cost in a given row
        #np.all(np.any(costs[:i]>c,axis=1)) -- evaluates this is true for all rows?
        #same for all rows after row i
        is_efficient[i] = np.all(np.any(costs[:i]<c, axis=1)) and np.all(np.any(costs[i+1:]<c, axis=1)) #changed to <
    return is_efficient

#### Test: is pareto efficient dumb

Give a set of input points, can change to test. 

In [ ]:
inputPoints = np.array([[1,0,0],[0,1,0],[0,0,1],[1,0,1]])

In [ ]:
result = is_pareto_efficient_dumb(inputPoints)
result

Test on 2d data

In [ ]:
#X cost sets with two types of cost/set
data=np.random.rand(2,10000)
plt.plot(data[0,:],data[1,:],'.b',label='Original Data')
plt.xlabel('Objective 1')
plt.ylabel('Object 2')
plt.legend()

In [ ]:
data = data.T

In [ ]:
result = is_pareto_efficient_dumb(data)

In [ ]:
data = data.tolist()
paretoPoints=[]
dominatedPoints=[]
for i in range(len(data)):
    if result[i] == True:
        paretoPoints.append(data[i])
    else:
        dominatedPoints.append(data[i])

In [ ]:
paretoPoints = np.array(paretoPoints)

In [ ]:
dominatedPoints = np.array(dominatedPoints)

In [ ]:
plt.plot(paretoPoints[:,0],paretoPoints[:,1],'.r',label='Pareto Points')
plt.plot(dominatedPoints[:,0],dominatedPoints[:,1],'.b',label='Dominated Points')
plt.xlabel('Objective 1')
plt.ylabel('Object 2')
plt.legend()

Test on higher dimensional data:

In [ ]:
data=np.random.rand(4,1000000)
data = data.T
result = is_pareto_efficient_dumb(data)

Observations:
    
    1. Seems to run quicly for 10000 points in 2d
    2. With 4 costs, runs quickly for 10000 datapoints
    3. Took a little longer with 100000 points
    4. >11 minutes with 1 mil points, 4 costs

### Is_pareto_efficient_simple

*This is when looking for the highest cost value in a set (need to change sign otherwise)

In [ ]:
def is_pareto_efficient_simple(costs):
    '''
    Function to find the pareto-efficient points, fast for many datapoints, but slower for multiple costs. Function updates its knowledge of efficiency after evaluating each row.
    
    Inputs:
    costs -- An (n_points, n_costs) array
    
    Output:
    is_efficient -- A (n_points, ) boolean array, indicating whether each point is Pareto efficient
    '''
    #Make an output array
    is_efficient = np.ones(costs.shape[0], dtype = bool)
    
    #for row i and cost value c in the costs array
    for i, c in enumerate(costs):
        #look at a specific row
        if is_efficient[i]:
            #check row against all others and assign them is_efficient values (True/False) dependin on if they have better/worse values than the current row, thus is efficient is continuously updated with new information
            is_efficient[is_efficient] = np.any(costs[is_efficient]>c, axis=1)  
            is_efficient[i] = True  ## And list index i as efficient... this will change as other row evaluations are done
       
    return is_efficient

#### Test: is pareto efficient simple

Give a set of input points, can change to test. 

In [ ]:
inputPoints = np.array([[1,0,0],[0,1,0],[0,0,1],[1,0,1]])

In [ ]:
result = is_pareto_efficient_simple(inputPoints)
result

Test on 2d data

In [ ]:
#X cost sets with two types of cost/set
data=np.random.rand(2,10000)
plt.plot(data[0,:],data[1,:],'.b',label='Original Data')
plt.xlabel('Objective 1')
plt.ylabel('Object 2')
plt.legend()

In [ ]:
data = data.T

In [ ]:
result = is_pareto_efficient_simple(data)

In [ ]:
data = data.tolist()
paretoPoints=[]
dominatedPoints=[]
for i in range(len(data)):
    if result[i] == True:
        paretoPoints.append(data[i])
    else:
        dominatedPoints.append(data[i])

In [ ]:
paretoPoints = np.array(paretoPoints)

In [ ]:
dominatedPoints = np.array(dominatedPoints)

In [ ]:
plt.plot(paretoPoints[:,0],paretoPoints[:,1],'.r',label='Pareto Points')
plt.plot(dominatedPoints[:,0],dominatedPoints[:,1],'.b',label='Dominated Points')
plt.xlabel('Objective 1')
plt.ylabel('Object 2')
plt.legend()

Test on higher dimensional points

In [ ]:
data=np.random.rand(4,1000000)
data = data.T
result = is_pareto_efficient_simple(data)

Observations:
    
    1. Seems to run quicly for 10000 points in 2d
    2. With 4 costs, runs quickly for 10000 datapoints
    3. Runs quickly with 100000 points
    4. Runs in seconds with 1 mil points, 4 costs

### Is_pareto_efficient

*This is when looking for the highest cost value in a set (need to change sign otherwise)

In [ ]:
def is_pareto_efficient(costs, return_mask = True):
    '''
    Function to find the pareto-efficient points, fast for many datapoints, but slower for multiple costs; faster than pareto efficient simple; constantly updates pareto information, moves fast because it removes previously dominated points from comparison
    
    Inputs:
    costs -- An (n_points, n_costs) array
    return_mask -- True to return a mask
    
    Output:
    is_efficient -- An array of indices of pareto-efficient points.
        *If return_mask is True, this will be an (n_points, ) boolean array
        Otherwise it will be a (n_efficient_points, ) integer array of indices.
    '''
    #make array of the indices of the costs [0,1,2...len(cost)]
    is_efficient = np.arange(costs.shape[0])
    
    #number of points (number of rows of costs array)
    n_points = costs.shape[0]
    
    next_point_index = 0  # Initialize next point counter
    
    #do until the input array has been searched through
    while next_point_index<len(costs):
        
        nondominated_point_mask = np.any(costs>costs[next_point_index], axis=1)#true/false array for whether the costs of the current row are greater/less than the costs in the other rows
        #assign true value for row being examined
        nondominated_point_mask[next_point_index] = True
        is_efficient = is_efficient[nondominated_point_mask]  # Apply non_dominated points mask to is_efficient array
        #costs/input file now contains only nondominated points so far
        costs = costs[nondominated_point_mask]
        #next point to examine is the sum of the T/F values up to the previous point plus 1
        next_point_index = np.sum(nondominated_point_mask[:next_point_index])+1
        
    #if the return maks is true, the output is a boolean array, otherwise it will just be the indices of the efficient points
    if return_mask:
        is_efficient_mask = np.zeros(n_points, dtype = bool)
        is_efficient_mask[is_efficient] = True
        return is_efficient_mask
    else:
        return is_efficient

#### Test: is pareto efficient 

Give a set of input points, can change to test. 

In [ ]:
inputPoints = np.array([[1,0,0],[0,1,0],[0,0,1],[1,0,1]])

In [ ]:
result = is_pareto_efficient(inputPoints)
result

Test on 2d data

In [ ]:
#X cost sets with two types of cost/set
data=np.random.rand(2,10000)
plt.plot(data[0,:],data[1,:],'.b',label='Original Data')
plt.xlabel('Objective 1')
plt.ylabel('Object 2')
plt.legend()

In [ ]:
data = data.T

In [ ]:
result = is_pareto_efficient(data)

In [ ]:
data = data.tolist()
paretoPoints=[]
dominatedPoints=[]
for i in range(len(data)):
    if result[i] == True:
        paretoPoints.append(data[i])
    else:
        dominatedPoints.append(data[i])

In [ ]:
paretoPoints = np.array(paretoPoints)

In [ ]:
dominatedPoints = np.array(dominatedPoints)

In [ ]:
plt.plot(paretoPoints[:,0],paretoPoints[:,1],'.r',label='Pareto Points')
plt.plot(dominatedPoints[:,0],dominatedPoints[:,1],'.b',label='Dominated Points')
plt.xlabel('Objective 1')
plt.ylabel('Object 2')
plt.legend()

Test on higher dimensional data

In [ ]:
data=np.random.rand(4,1000000)
data = data.T
result = is_pareto_efficient(data)

Observations:
    
    1. Seems to run quicly for 10000 points in 2d
    2. With 4 costs, runs quickly for 10000 datapoints
    3. Runs quickly with 100000 points
    4. Runs in seconds with 1 mil points, 4 costs

### Test all against the same dataset

#### 2D data (100 points)

In [ ]:
data_2D=np.random.rand(2,10)
data_SC = data_2D
data_dumb = data_2D
data_simple = data_2D
data_efficient = data_2D

In [ ]:
plt.plot(data_2D[0,:],data_2D[1,:],'.b',label='Original Data')
plt.xlabel('Objective 1')
plt.ylabel('Object 2')
plt.legend()

##### Simple cull

In [ ]:
data_SC = data_SC.T
data_list_SC = data_SC.tolist()
paretoPoints_SC, dominatedPoints_SC = simple_cull(data_list_SC, dominates)
paretoPoints_SC = np.array(list(paretoPoints_SC))
dominatedPoints_SC = np.array(list(dominatedPoints_SC))
plt.plot(paretoPoints_SC[:,0],paretoPoints_SC[:,1],'.r',label='Pareto Points')
plt.plot(dominatedPoints_SC[:,0],dominatedPoints_SC[:,1],'.b',label='Dominated Points')
plt.xlabel('Objective 1')
plt.ylabel('Object 2')
plt.title('Simple Cull')
plt.legend()
print("Pareto Optimal Points:",paretoPoints_SC)

##### Is pareto efficient dumb

In [ ]:
data_dumb=data_dumb.T
result = is_pareto_efficient_dumb(data_dumb)

In [ ]:
data_list_dumb = data_dumb.tolist()
paretoPoints=[]
dominatedPoints=[]
for i in range(len(data_list_dumb)):
    if result[i] == True:
        paretoPoints.append(data_list_dumb[i])
    else:
        dominatedPoints.append(data_list_dumb[i])

In [ ]:
paretoPoints = np.array(paretoPoints)
dominatedPoints = np.array(dominatedPoints)

In [ ]:
plt.plot(paretoPoints[:,0],paretoPoints[:,1],'.r',label='Pareto Points')
plt.plot(dominatedPoints[:,0],dominatedPoints[:,1],'.b',label='Dominated Points')
plt.xlabel('Objective 1')
plt.ylabel('Object 2')
plt.title('Pareto Efficient Dumb')
plt.legend()
print("Pareto Optimal Points:",paretoPoints)

##### is pareto efficient simple

In [ ]:
data_simple = data_simple.T
result = is_pareto_efficient_simple(data_simple)
data_list_simple = data_simple.tolist()
paretoPoints=[]
dominatedPoints=[]
for i in range(len(data_list_simple)):
    if result[i] == True:
        paretoPoints.append(data_list_simple[i])
    else:
        dominatedPoints.append(data_list_simple[i])
paretoPoints = np.array(paretoPoints)
dominatedPoints = np.array(dominatedPoints)
plt.plot(paretoPoints[:,0],paretoPoints[:,1],'.r',label='Pareto Points')
plt.plot(dominatedPoints[:,0],dominatedPoints[:,1],'.b',label='Dominated Points')
plt.xlabel('Objective 1')
plt.ylabel('Object 2')
plt.title('Pareto Efficient Simple')
plt.legend()
print("Pareto Optimal Points:",paretoPoints)

##### is pareto efficient 

In [ ]:
data_efficient = data_efficient.T
result = is_pareto_efficient(data_efficient)
data_list_efficient = data_efficient.tolist()
paretoPoints=[]
dominatedPoints=[]
for i in range(len(data_list_efficient)):
    if result[i] == True:
        paretoPoints.append(data_list_efficient[i])
    else:
        dominatedPoints.append(data_list_efficient[i])
paretoPoints = np.array(paretoPoints)
dominatedPoints = np.array(dominatedPoints)
plt.plot(paretoPoints[:,0],paretoPoints[:,1],'.r',label='Pareto Points')
plt.plot(dominatedPoints[:,0],dominatedPoints[:,1],'.b',label='Dominated Points')
plt.xlabel('Objective 1')
plt.ylabel('Object 2')
plt.title('Pareto Efficient')
plt.legend()
print("Pareto Optimal Points:",paretoPoints)

#### 4D data (100 points)

In [ ]:
data_4D=np.random.rand(4,100)
data_SC = data_4D
data_dumb = data_4D
data_simple = data_4D
data_efficient = data_4D

##### Simple Cull

In [ ]:
start = timeit.default_timer()
data_SC = data_SC.T
data_list_SC = data_SC.tolist()
paretoPoints, dominatedPoints = simple_cull(data_list_SC, dominates)
paretoPoints_SC = np.array(list(paretoPoints))
dominatedPoints_SC = np.array(list(dominatedPoints))
stop = timeit.default_timer()

print("Number of Pareto Optimal Points", len(paretoPoints_SC))

print("Pareto Optimal Points:",paretoPoints_SC)

print('Time:',stop-start)

##### Pareto Dumb

In [ ]:
start = timeit.default_timer()
data_dumb=data_dumb.T
result = is_pareto_efficient_dumb(data_dumb)
data_list_dumb = data_dumb.tolist()
paretoPoints=[]
dominatedPoints=[]
for i in range(len(data_list_dumb)):
    if result[i] == True:
        paretoPoints.append(data_list_dumb[i])
    else:
        dominatedPoints.append(data_list_dumb[i])
paretoPoints_dumb = np.array(paretoPoints)
dominatedPoints = np.array(dominatedPoints)
stop = timeit.default_timer()

print("Number of Pareto Optimal Points", len(paretoPoints_dumb))

print("Pareto Optimal Points:",paretoPoints_dumb)

print('Time:',stop-start)

##### Pareto Simple

In [ ]:
start = timeit.default_timer()
data_simple = data_simple.T
result = is_pareto_efficient_simple(data_simple)
data_list_simple = data_simple.tolist()
paretoPoints=[]
dominatedPoints=[]
for i in range(len(data_list_simple)):
    if result[i] == True:
        paretoPoints.append(data_list_simple[i])
    else:
        dominatedPoints.append(data_list_simple[i])
paretoPoints_simple = np.array(paretoPoints)
dominatedPoints = np.array(dominatedPoints)
stop = timeit.default_timer()

print("Number of Pareto Optimal Points", len(paretoPoints_simple))

print("Pareto Optimal Points:",paretoPoints_simple)

print('Time:',stop-start)

##### Pareto Efficient

In [ ]:
start = timeit.default_timer()
data_efficient = data_efficient.T
result = is_pareto_efficient(data_efficient)
data_list_efficient = data_efficient.tolist()
paretoPoints=[]
dominatedPoints=[]
for i in range(len(data_list_efficient)):
    if result[i] == True:
        paretoPoints.append(data_list_efficient[i])
    else:
        dominatedPoints.append(data_list_efficient[i])
paretoPoints_efficient = np.array(paretoPoints)
dominatedPoints = np.array(dominatedPoints)
stop = timeit.default_timer()

print("Number of Pareto Optimal Points", len(paretoPoints_efficient))

print("Pareto Optimal Points:",paretoPoints_efficient)

print('Time:',stop-start)

##### Compare pareto sets

In [ ]:
compare_pareto_sets(paretoPoints_efficient,paretoPoints_dumb)

Observations:
    1. From spot checking, simple cull method gives the same pareto set, but in a different order, so can't use compare_pareto_sets function
    2. All functions take similar time

#### 4D data (1000 points)

In [ ]:
data_4D_1k=np.random.rand(4,1000)
data_SC = data_4D_1k
data_dumb = data_4D_1k
data_simple = data_4D_1k
data_efficient = data_4D_1k

##### Simple Cull

In [ ]:
start = timeit.default_timer()
data_SC = data_SC.T
data_list_SC = data_SC.tolist()
paretoPoints, dominatedPoints = simple_cull(data_list_SC, dominates)
paretoPoints_SC = np.array(list(paretoPoints))
dominatedPoints_SC = np.array(list(dominatedPoints))
stop = timeit.default_timer()

print("Number of Pareto Optimal Points", len(paretoPoints_SC))

print("Pareto Optimal Points:",paretoPoints_SC)

print('Time:',stop-start)

##### Pareto Dumb

In [ ]:
start = timeit.default_timer()
data_dumb=data_dumb.T
result = is_pareto_efficient_dumb(data_dumb)
data_list_dumb = data_dumb.tolist()
paretoPoints=[]
dominatedPoints=[]
for i in range(len(data_list_dumb)):
    if result[i] == True:
        paretoPoints.append(data_list_dumb[i])
    else:
        dominatedPoints.append(data_list_dumb[i])
paretoPoints_dumb = np.array(paretoPoints)
dominatedPoints = np.array(dominatedPoints)
stop = timeit.default_timer()

print("Number of Pareto Optimal Points", len(paretoPoints_dumb))

print("Pareto Optimal Points:",paretoPoints_dumb)

print('Time:',stop-start)

##### Pareto Simple

In [ ]:
start = timeit.default_timer()
data_simple = data_simple.T
result = is_pareto_efficient_simple(data_simple)
data_list_simple = data_simple.tolist()
paretoPoints=[]
dominatedPoints=[]
for i in range(len(data_list_simple)):
    if result[i] == True:
        paretoPoints.append(data_list_simple[i])
    else:
        dominatedPoints.append(data_list_simple[i])
paretoPoints_simple = np.array(paretoPoints)
dominatedPoints = np.array(dominatedPoints)
stop = timeit.default_timer()

print("Number of Pareto Optimal Points", len(paretoPoints_simple))

print("Pareto Optimal Points:",paretoPoints_simple)

print('Time:',stop-start)

##### Pareto Efficient

In [ ]:
start = timeit.default_timer()
data_efficient = data_efficient.T
result = is_pareto_efficient(data_efficient)
data_list_efficient = data_efficient.tolist()
paretoPoints=[]
dominatedPoints=[]
for i in range(len(data_list_efficient)):
    if result[i] == True:
        paretoPoints.append(data_list_efficient[i])
    else:
        dominatedPoints.append(data_list_efficient[i])
paretoPoints_efficient = np.array(paretoPoints)
dominatedPoints = np.array(dominatedPoints)
stop = timeit.default_timer()

print("Number of Pareto Optimal Points", len(paretoPoints_efficient))

print("Pareto Optimal Points:",paretoPoints_efficient)

print('Time:',stop-start)

##### Compare Pareto Sets

In [ ]:
compare_pareto_sets(paretoPoints_efficient,paretoPoints_dumb)

Observations:
    1. From spot checking, simple cull method gives the same pareto set, but in a different order, so can't use compare_pareto_sets function
    2. All functions take similar time
    3. As noted above, run times are trivial until looking at 1million points, at which point the pareto simple and pareto efficient functions still work

#### 2D data (100 points) -- min costs

Test just pareto simple and pareto efficient

In [ ]:
from pareto_utils import(
    compare_pareto_sets,
    is_pareto_efficient_simple,
    is_pareto_efficient)

In [ ]:
data_2D_min=np.random.rand(2,100)
data_simple_min = data_2D_min
data_efficient_min = data_2D_min

In [ ]:
plt.plot(data_2D_min[0,:],data_2D_min[1,:],'.b',label='Original Data')
plt.xlabel('Objective 1')
plt.ylabel('Object 2')
plt.legend()

##### Pareto simple

In [ ]:
data_simple_min = data_simple_min.T
result = is_pareto_efficient_simple(data_simple_min)
data_list_simple_min = data_simple_min.tolist()
paretoPoints=[]
dominatedPoints=[]
for i in range(len(data_list_simple_min)):
    if result[i] == True:
        paretoPoints.append(data_list_simple_min[i])
    else:
        dominatedPoints.append(data_list_simple_min[i])
paretoPoints_simple = np.array(paretoPoints)
dominatedPoints = np.array(dominatedPoints)
plt.plot(paretoPoints_simple[:,0],paretoPoints_simple[:,1],'.r',label='Pareto Points')
plt.plot(dominatedPoints[:,0],dominatedPoints[:,1],'.b',label='Dominated Points')
plt.xlabel('Objective 1')
plt.ylabel('Object 2')
plt.title('Pareto Efficient Simple')
plt.legend()
print("Number of Pareto Optimal Points:",len(paretoPoints_simple))
print("Pareto Optimal Points:",paretoPoints_simple)

##### Pareto efficient

In [ ]:
data_efficient_min = data_efficient_min.T
result = is_pareto_efficient(data_efficient_min)
data_list_efficient_min = data_efficient_min.tolist()
paretoPoints=[]
dominatedPoints=[]
for i in range(len(data_list_efficient_min)):
    if result[i] == True:
        paretoPoints.append(data_list_efficient_min[i])
    else:
        dominatedPoints.append(data_list_efficient_min[i])
paretoPoints_efficient = np.array(paretoPoints)
dominatedPoints = np.array(dominatedPoints)
plt.plot(paretoPoints_efficient[:,0],paretoPoints_efficient[:,1],'.r',label='Pareto Points')
plt.plot(dominatedPoints[:,0],dominatedPoints[:,1],'.b',label='Dominated Points')
plt.xlabel('Objective 1')
plt.ylabel('Object 2')
plt.title('Pareto Efficient')
plt.legend()
print("Number of Pareto Optimal Points:",len(paretoPoints_efficient))
print("Pareto Optimal Points:",paretoPoints_efficient)

##### Compare pareto sets

In [ ]:
compare_pareto_sets(paretoPoints_efficient,paretoPoints_simple)

#### Test find_pareto_set function

In [ ]:
from pareto_utils import(
    compare_pareto_sets,
    is_pareto_efficient_simple,
    is_pareto_efficient,
    find_pareto_set)

#### 2D data (100 points) -- min costs

In [ ]:
data_2D_min=np.random.rand(2,100)
data_simple_min = data_2D_min
data_efficient_min = data_2D_min

In [ ]:
plt.plot(data_2D_min[0,:],data_2D_min[1,:],'.b',label='Original Data')
plt.xlabel('Objective 1')
plt.ylabel('Object 2')
plt.legend()

In [ ]:
data_simple_min = data_simple_min.T
result,paretoPoints,dominatedPoints=find_pareto_set(data_simple_min,is_pareto_efficient_simple)
plt.plot(paretoPoints[:,0],paretoPoints[:,1],'.r',label='Pareto Points')
plt.plot(dominatedPoints[:,0],dominatedPoints[:,1],'.b',label='Dominated Points')
plt.xlabel('Objective 1')
plt.ylabel('Object 2')
plt.title('Pareto Efficient Simple')
plt.legend()
print("Number of Pareto Optimal Points:",len(paretoPoints))
print("Pareto Optimal Points:",paretoPoints)
print("Dominated Points:",dominatedPoints)

In [ ]:
data_efficient_min = data_efficient_min.T
result,paretoPoints,dominatedPoints=find_pareto_set(data_efficient_min,is_pareto_efficient_simple)
plt.plot(paretoPoints[:,0],paretoPoints[:,1],'.r',label='Pareto Points')
plt.plot(dominatedPoints[:,0],dominatedPoints[:,1],'.b',label='Dominated Points')
plt.xlabel('Objective 1')
plt.ylabel('Object 2')
plt.title('Pareto Efficient Simple')
plt.legend()
print("Number of Pareto Optimal Points:",len(paretoPoints))
print("Pareto Optimal Points:",paretoPoints)
print("Dominated Points:",dominatedPoints)

#### 2D data (100 points) -- max costs

In [ ]:
data_2D_min=np.random.rand(2,100)
data_simple_min = data_2D_min
data_efficient_min = data_2D_min

In [ ]:
plt.plot(data_2D_min[0,:],data_2D_min[1,:],'.b',label='Original Data')
plt.xlabel('Objective 1')
plt.ylabel('Object 2')
plt.legend()

In [ ]:
data_simple_min = data_simple_min.T
result,paretoPoints,dominatedPoints=find_pareto_set(data_simple_min,is_pareto_efficient_simple,True)
plt.plot(paretoPoints[:,0],paretoPoints[:,1],'.r',label='Pareto Points')
plt.plot(dominatedPoints[:,0],dominatedPoints[:,1],'.b',label='Dominated Points')
plt.xlabel('Objective 1')
plt.ylabel('Object 2')
plt.title('Pareto Efficient Simple')
plt.legend()
print("Number of Pareto Optimal Points:",len(paretoPoints))
print("Pareto Optimal Points:",paretoPoints)
print("Dominated Points:",dominatedPoints)

In [ ]:
data_efficient_min = data_efficient_min.T
result,paretoPoints,dominatedPoints=find_pareto_set(data_efficient_min,is_pareto_efficient_simple,True)
plt.plot(paretoPoints[:,0],paretoPoints[:,1],'.r',label='Pareto Points')
plt.plot(dominatedPoints[:,0],dominatedPoints[:,1],'.b',label='Dominated Points')
plt.xlabel('Objective 1')
plt.ylabel('Object 2')
plt.title('Pareto Efficient Simple')
plt.legend()
print("Number of Pareto Optimal Points:",len(paretoPoints))
print("Pareto Optimal Points:",paretoPoints)
print("Dominated Points:",dominatedPoints)